In [1]:
import os
from os import path
import pandas as pd
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
warnings.filterwarnings('ignore')
sns.set_theme(style="white")



In [2]:
path = 'PAN/Data/'
paths = [path +csv for csv in os.listdir(path)]
CLEAN = [path for path in paths if 'CLN' in path]

In [3]:
data = pd.read_csv(CLEAN[0])
FEATS = list(data.columns[:9])
FEATS.remove('i_qnt')

LABELS = ['0.5','0.75']

In [4]:
HALF = [path for path in CLEAN if "_50_" in path]
SEVE = [path for path in CLEAN if "_75_" in path]
NINE = [path for path in CLEAN if "_90_" in path]

The next line will verify whether all the columns are in the same order across all DataFrames

In [6]:
FEATS_TABLE = pd.read_csv(HALF[0])[FEATS]

flag = True
for path in CLEAN:
    data = pd.read_csv(path)[FEATS]
    for i in range(data.shape[0]):
        if not data.iloc[i].equals(FEATS_TABLE.iloc[i]): 
            flag = False
flag

True

In [8]:
from os import path

path_50 = 'PAN/ALL/CLN_HLT_ALL_50_qnt.csv'

if not path.exists(path_50):
    
    LABELS_HALF = [label for label in LABELS]*3
    LABELS_HALF = [label+'_'+ds[-14:-11] for label,ds in zip(LABELS_HALF,HALF*3)]
    
    CLN_HLT_ALL_50_qnt = pd.DataFrame(columns=FEATS+LABELS_HALF)
    TTI_50 = pd.read_csv(HALF[0])
    TTO_50 = pd.read_csv(HALF[1])
    WOP_50 = pd.read_csv(HALF[2])
    
    for i in range(TTI_50.shape[0]):
        FEATS_ = {feat:value for feat,value in zip(FEATS, TTI_50[FEATS].iloc[i])}
        LABELS_ = {}
        for label in LABELS_HALF:
            if "TTI" in label:
                LABELS_[label] = TTI_50[label[:-4]].iloc[i]
            elif "TTO" in label:
                LABELS_[label] = TTO_50[label[:-4]].iloc[i]
            elif "WOP" in label:
                LABELS_[label] = WOP_50[label[:-4]].iloc[i]

        FEATS_.update(LABELS_)
        CLN_HLT_ALL_50_qnt.loc[i] = FEATS_


    CLN_HLT_ALL_50_qnt.to_csv(path_50)

In [10]:
path_75 = 'PAN/ALL/CLN_HLT_ALL_75_qnt.csv'

if not path.exists(path_75):
    
    LABELS_SEVE = [label for label in LABELS]*3
    LABELS_SEVE = [label+'_'+ds[-14:-11] for label,ds in zip(LABELS_SEVE,SEVE*3)]
    
    CLN_HLT_ALL_75_qnt = pd.DataFrame(columns=FEATS+LABELS_SEVE)
    TTI_75 = pd.read_csv(SEVE[0])
    TTO_75 = pd.read_csv(SEVE[1])
    WOP_75 = pd.read_csv(SEVE[2])
    
    for i in range(TTI_75.shape[0]):
        FEATS_ = {feat:value for feat,value in zip(FEATS, TTI_75[FEATS].iloc[i])}
        LABELS_ = {}
        for label in LABELS_SEVE:
            if "TTI" in label:
                LABELS_[label] = TTI_75[label[:-4]].iloc[i]
            elif "TTO" in label:
                LABELS_[label] = TTO_75[label[:-4]].iloc[i]
            elif "WOP" in label:
                LABELS_[label] = WOP_75[label[:-4]].iloc[i]
                
        FEATS_.update(LABELS_)
        CLN_HLT_ALL_75_qnt.loc[i] = FEATS_


    CLN_HLT_ALL_75_qnt.to_csv(path_75)

In [11]:
path_90 = 'PAN/ALL/CLN_HLT_ALL_90_qnt.csv'

if not path.exists(path_90):
    
    LABELS_NINE = [label for label in LABELS]*3
    LABELS_NINE = [label+'_'+ds[-14:-11] for label,ds in zip(LABELS_NINE,NINE*3)]
    
    CLN_HLT_ALL_90_qnt = pd.DataFrame(columns=FEATS+LABELS_NINE)
    TTI_90 = pd.read_csv(NINE[0])
    TTO_90 = pd.read_csv(NINE[1])
    WOP_90 = pd.read_csv(NINE[2])
    
    for i in range(TTI_90.shape[0]):
        FEATS_ = {feat:value for feat,value in zip(FEATS, TTI_90[FEATS].iloc[i])}
        LABELS_ = {}
        for label in LABELS_NINE:
            if "TTI" in label:
                LABELS_[label] = TTI_90[label[:-4]].iloc[i]
            elif "TTO" in label:
                LABELS_[label] = TTO_90[label[:-4]].iloc[i]
            elif "WOP" in label:
                LABELS_[label] = WOP_90[label[:-4]].iloc[i]

        FEATS_.update(LABELS_)
        CLN_HLT_ALL_90_qnt.loc[i] = FEATS_


    CLN_HLT_ALL_90_qnt.to_csv(path_90)

In [12]:
CLN_HLT_ALL_75_qnt

,i_smx,i_sgv,i_sgn,i_rsg,i_rer,i_ren,i_gsv,i_fic,0.5_TTO,0.75_WOP,0.5_TTI,0.75_TTO,0.5_WOP,0.75_TTI
0,True,False,False,0.0000,0.0,0,0.000000,0.0,0,4,4,0,4,4
1,True,False,False,0.0000,0.0,0,0.000043,0.0,0,4,4,0,4,4
2,True,False,False,0.0000,0.0,0,0.000157,0.0,0,4,4,0,4,4
3,True,False,False,0.0000,0.0,0,0.007563,0.0,0,4,4,0,4,4
4,True,False,False,0.0000,0.0,0,0.000000,0.1,0,4,4,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46459,False,False,True,0.0001,1.0,10,0.007563,0.9,0,0,0,0,0,0
46460,False,False,True,0.0001,1.0,10,0.000000,1.0,0,0,0,1,0,0
46461,False,False,True,0.0001,1.0,10,0.000043,1.0,0,0,0,0,0,0
46462,False,False,True,0.0001,1.0,10,0.000157,1.0,0,0,0,0,0,0


In [13]:
CLN_HLT_ALL_90_qnt

,i_smx,i_sgv,i_sgn,i_rsg,i_rer,i_ren,i_gsv,i_fic,0.5_TTI,0.75_TTO,0.5_WOP,0.75_TTI,0.5_TTO,0.75_WOP
0,True,False,False,0.0000,0.0,0,0.000000,0.0,4,4,0,4,4,0
1,True,False,False,0.0000,0.0,0,0.000043,0.0,4,4,0,4,4,0
2,True,False,False,0.0000,0.0,0,0.000157,0.0,4,4,0,4,4,0
3,True,False,False,0.0000,0.0,0,0.007563,0.0,4,4,0,4,4,0
4,True,False,False,0.0000,0.0,0,0.000000,0.1,4,4,0,4,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46459,False,False,True,0.0001,1.0,10,0.007563,0.9,0,0,0,0,0,0
46460,False,False,True,0.0001,1.0,10,0.000000,1.0,0,0,0,0,0,1
46461,False,False,True,0.0001,1.0,10,0.000043,1.0,0,0,0,0,0,0
46462,False,False,True,0.0001,1.0,10,0.000157,1.0,0,0,0,0,0,0
